# Azimuth Max Velocity Errors
Craig Lage - 17-Oct-25

In [ ]:
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from astropy.time import Time, TimeDelta
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst_efd_client import merge_packed_time_series as mpts
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
client = makeEfdClient()

In [ ]:
# First, find the times of the error
start = Time("2025-10-28T12:00")
end = Time("2025-10-29T12:00")
errors = getEfdData(client, "lsst.sal.ATMCS.logevent_errorCode", begin=start, end=end)
reports = errors['errorReport'].values
times = errors['private_kafkaStamp'].values
fails = []
pdf = PdfPages(f"/home/cslage/DATA/Max_Velocity_Errors_7_17Oct25.pdf")
for i, report in enumerate(reports):
    # For each error, only choose the errors we are interested in
    if ("max velocity error" in report) or ("max tachometer difference" in report):
        print(report, Time(times[i], format='unix_tai').utc.isot)
        fails.append(times[i])
for fail in fails:
        # For each error, get the mount data and plot it.
        fail = Time(fail, format='unix_tai').utc
        t_start = fail - TimeDelta(20.0, format='sec')
        t_end = fail + TimeDelta(1.0, format='sec')
        print(t_start.isot, t_end.isot, fail.isot)
        
        mount_position = getEfdData(client, "lsst.sal.ATMCS.mount_AzEl_Encoders", begin=t_start, end=t_end)
        nasmyth_position = getEfdData(client, "lsst.sal.ATMCS.mount_Nasmyth_Encoders", begin=t_start, end=t_end)
        torques = getEfdData(client, "lsst.sal.ATMCS.measuredTorque", begin=t_start, end=t_end)
        vels = getEfdData(client, "lsst.sal.ATMCS.measuredMotorVelocity", begin=t_start, end=t_end)
        torque_dem = getEfdData(client, "lsst.sal.ATMCS.torqueDemand", begin=t_start, end=t_end)
        az = mpts(mount_position, 'azimuthCalculatedAngle', stride=1)
        if abs(Time(az.index[0]).unix_tai - Time(t_start).unix_tai) > 10.0:
            continue
        azEnc1 = mpts(mount_position, 'azimuthEncoder1Raw', stride=1)
        azEnc2 = mpts(mount_position, 'azimuthEncoder2Raw', stride=1)
        azEnc3 = mpts(mount_position, 'azimuthEncoder3Raw', stride=1)
        el = mpts(mount_position, 'elevationCalculatedAngle', stride=1)
        rot = mpts(nasmyth_position, 'nasmyth2CalculatedAngle', stride=1)
        az_torque_1 = mpts(torques, 'azimuthMotor1Torque', stride=1)
        az_torque_2 = mpts(torques, 'azimuthMotor2Torque', stride=1)
        az_torque_dem_1 = mpts(torque_dem, 'azimuthMotor1Torque', stride=1)
        az_torque_dem_2 = mpts(torque_dem, 'azimuthMotor2Torque', stride=1)
        el_torque = mpts(torques, 'elevationMotorTorque', stride=1)
        rot_torque = mpts(torques, 'nasmyth2MotorTorque', stride=1)
        az_vel_1 = mpts(vels, 'azimuthMotor1Velocity', stride=1)
        az_vel_2 = mpts(vels, 'azimuthMotor2Velocity', stride=1)
    
        fig, axs = plt.subplots(3,1,figsize = (10,12))
        plt.suptitle(f"Max velocity error {fail.isot}", fontsize=18)
        axs[0].set_title("Torques")
        az_torque_1['azimuthMotor1Torque'].plot(ax = axs[0], label="Torque1")
        az_torque_2['azimuthMotor2Torque'].plot(ax = axs[0], label="Torque2")
        axs[0].axvline(fail.isot, ls='--', color='red', label='Logged fail')
        axs[0].legend()
        axs[0].set_xticks([])
        
        axs[1].set_title("Velocities")
        az_vel_1['azimuthMotor1Velocity'].plot(ax = axs[1], label="Vel1")
        az_vel_2['azimuthMotor2Velocity'].plot(ax = axs[1], label="Vel2")
        axs[1].axvline(fail.isot, ls='--', color='red', label='Logged fail')
        axs[1].legend()
        axs[1].set_xticks([])
        axs[2].set_title("Azimuth")
        az['azimuthCalculatedAngle'].plot(ax = axs[2], label='Azimuth')
        axs[2].axvline(fail.isot, ls='--', color='red', label='Logged fail')
        axs[2].legend()
        axs[2].xaxis.set_major_locator(mdates.SecondLocator(interval=1))
        axs[2].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
        plt.xticks(rotation=45)
        pdf.savefig(fig)  # saves the current figure into a pdf page
        plt.clf()
pdf.close()
